In [24]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


In [16]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)


In [3]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)

pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')

Generation 1 - Current best internal CV score: 0.9799476800220294
Generation 2 - Current best internal CV score: 0.9799476800220294
Generation 3 - Current best internal CV score: 0.9799476800220294
Generation 4 - Current best internal CV score: 0.9799476800220294
Generation 5 - Current best internal CV score: 0.9799559410711828

Best pipeline: ExtraTreesClassifier(MaxAbsScaler(input_matrix), bootstrap=False, criterion=gini, max_features=0.3, min_samples_leaf=3, min_samples_split=4, n_estimators=100)
0.9755555555555555


In [28]:
!cat tpot_digits_pipeline.py

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.9710422690348341
exported_pipeline = LogisticRegression(C=0.1, dual=False, penalty="l2")

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


# configurations

In [17]:
# TPOT MDR

tpot_mdr_classifier_config_dict = {

    # Classifiers
    'mdr.MDRClassifier': {
        'tie_break': [0, 1],
        'default_label': [0, 1]
    },

    # Feature Selectors
    'skrebate.ReliefF': {
        'n_features_to_select': range(1, 6),
        'n_neighbors': [2, 10, 50, 100, 250, 500]
    },

    'skrebate.SURF': {
        'n_features_to_select': range(1, 6)
    },

    'skrebate.SURFstar': {
        'n_features_to_select': range(1, 6)
    },

    'skrebate.MultiSURF': {
        'n_features_to_select': range(1, 6)
    }
}

In [18]:
tpot_mdr_classifier_config_dict

{'mdr.MDRClassifier': {'tie_break': [0, 1], 'default_label': [0, 1]},
 'skrebate.ReliefF': {'n_features_to_select': range(1, 6),
  'n_neighbors': [2, 10, 50, 100, 250, 500]},
 'skrebate.SURF': {'n_features_to_select': range(1, 6)},
 'skrebate.SURFstar': {'n_features_to_select': range(1, 6)},
 'skrebate.MultiSURF': {'n_features_to_select': range(1, 6)}}

In [22]:
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,
                      config_dict="TPOT MDR")

tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_digits_pipeline_mdr.py')



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: LogisticRegression(CombineDFs(input_matrix, input_matrix), C=0.001, dual=False, penalty=l2)
0.9688888888888889


In [25]:
!cat tpot_digits_pipeline_mdr.py

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.9643563265868098
exported_pipeline = make_pipeline(
    make_union(
        FunctionTransformer(copy),
        FunctionTransformer(copy)
    ),
    LogisticRegression(C=0.001, dual=False, penalty="l2")
)

exported_pipeline.fit(training_features, training_target)
results 